<a href="https://colab.research.google.com/github/Benchang101/gdp-dashboard/blob/main/Chat_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers accelerate

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


model_name = "EleutherAI/gpt-neo-2.7B"  # AI model name
print("載入模型與分詞器中，可能需要一些時間...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# set pad_token using eos_token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# set GPU or CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用設備：{device}")
model = model.to(device)

# summon reply
def chat_with_ai(user_input):
    """
    與 AI 聊天的主要函式
    """
    # input
    input_text = f"User: {user_input}\nAI:"
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True).to(device)

    # summon reply
    outputs = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=100,  # word limit
        pad_token_id=tokenizer.pad_token_id,
        temperature=0.5,
        top_k=50,
        top_p=0.9,
        do_sample=True,
        repetition_penalty=2.0,  # penalty
        no_repeat_ngram_size=3,  # reduce repeat
    )

    # decode
    response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # response only after "AI:"
    if "AI:" in response:
        response = response.split("AI:")[-1].strip()

    # ensure response
    response_lines = response.split("\n")
    clean_response = response_lines[0] if response_lines else response

    return clean_response

# main body
def main():
    print("ChatGPT 模擬器已啟動！輸入 'quit' 退出。")
    while True:
        try:
            user_input = input("你: ")
            if user_input.lower() == "quit":
                print("退出 ChatGPT 模擬器。")
                break

            ai_response = chat_with_ai(user_input)
            print(f"AI: {ai_response}")

        except KeyboardInterrupt:
            print("\n退出 ChatGPT 模擬器。")
            break
        except Exception as e:
            print(f"發生錯誤：{e}")
            break

if __name__ == "__main__":
    main()